In [94]:
from Bio import pairwise2 as pw

In [9]:
from Bio import Seq

In [10]:
from Bio import SeqIO 
import Bio

In [11]:
from Bio.Alphabet import IUPAC

In [12]:
import numpy as np
import pandas as pd

In [ ]:
import pandas as pd

In [17]:
uniprot = list(SeqIO.parse('uniprot-proteome_human.fasta','fasta'))

In [18]:
for seq in uniprot:   #
    x = seq.id
    seq.id = x.split("|")[1]

In [21]:
ids = [seq.id for seq in uniprot]
names = [seq.name for seq in uniprot]

In [40]:
uni_dict = {}
for i in uniprot:
    uni_dict[i.id] =i

In [33]:
grist = pd.read_table('gristone_positive_data.txt')

In [41]:
grist.head(5)

,uniport_id,peptide,left_flanking,right_flanking,sample_id,ensembl_gene_id,express_sum_tpm,family,A1,B1,C1,A2,B2,C2,label
0,P62424,KVAPAPAVVKK,KAKGK,QEAKK,train_sample_28,ENSG00000148303,397.13,PTHR23105_SF29,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1
1,germline-TAOK3-NM_016281-140-G-to-A-S47N-Ser47...,HTNEVVAIKK,FATNA,MSYSG,train_sample_28,ENSG00000135090,21.64,PTHR24361_SF187,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1
2,Q460N5,RLLPGNATISK,VKREG,AGKLP,train_sample_28,ENSG00000173193,26.23,PTHR14453_SF53,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1
3,O15511,ALKNPPINTK,AALQA,SQAVK,train_sample_28,ENSG00000162704,79.10,PTHR12644_SF1,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1
4,P09234,MPVGPAPGM,PPPGM,RPPMG,train_sample_28,ENSG00000124562,40.97,PTHR31148_SF1,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1


In [93]:
def calc_identity_score(seq1,seq2,gap=-0.5,extend=-0.1):
    """
    return seq1 seq1 pairwise identiy score
    seq1 is positive 
    seq1 seq2 is string

    Bio.pairwise2.format_alignment output:
    MPKGKKAKG------
      |||||||      
    --KGKKAKGKKVAPA
      Score=7
      
    alignment output: [('MPKGKKAKG------', '--KGKKAKGKKVAPA', 7.0, 0, 15)]
    score = ali[0][2] = 7
    """
    ali = pw.align.globalxs(seq1,seq2,gap,extend,score_only=True)
    # gap penalty = -0.5 in case of cak caak score =3
    return ali/min(len(seq1),len(seq2)) # 返回短序列的值,防止substring

In [128]:
def window(seq, window_lenth=7,step=1):
    """
    return list of seq window slide
    seq is string
    """
    return [seq[i:i+window_lenth] for i in range(0,len(seq)-window_lenth+1,step)]

In [47]:
from Bio.pairwise2 import format_alignment

In [49]:
print(format_alignment(*ali[0]))

MPKGKKAKG------
  |||||||      
--KGKKAKGKKVAPA
  Score=7



In [96]:
import re

In [116]:
def search_all(pattern,string, flags=0):
    """
    return all matched pattern index
    """
    res=[]
    while len(string)>0:
        sobj=re.search(pattern, string, flags)
        if not sobj:
            break
        span=sobj.span()
        res.append((span[0],span[1]))
        string=string[span[0]+1:]
    return res

In [143]:
def flat(nums):
    res = []
    for i in nums:
        if isinstance(i, list):
            res.extend(flat(i))
        else:
            res.append(i)
    return res

In [167]:
def slide_with_flank(seq,full,step=1,up_flank=6,down_flank=6,flags=0):
    """
    return window slide result as list for a full str given potential seq and it's flank removed
    seq=abc, full = 01234abc45678abc1234 up=1 down=1 step =1
    result is ['012', '123', '567', '234']
    """
    res = []
    window_len = len(seq)

    if len(full) >= window_len:
        sobj = re.search(seq,full,flags)
        if not sobj: ## 没找到seq 直接返回slide 
            res.append(window(full,window_lenth=window_len,step=step))
            
        else:
            span = sobj.span()
            if (span[0]-up_flank>0) and (len(full[0:span[0]-up_flank]) >= window_len) :
                # 上游剩下的区域长度大于等于seq的长度
                res.append(window(full[0:span[0]-up_flank],window_lenth=window_len))
            #下游max(up,down)+window_lenth-1的full中有seq的话去掉
            #if
            if (span[1]+down_flank<len(full)) and (len(full[span[1]+down_flank:]) >= window_len): 
                # 下游剩下的区域长度大于等于seq的长度
                res.append(slide_with_flank(seq,full[span[1]+down_flank:],step=step,
                           up_flank=up_flank,down_flank=down_flank,flags=flags))
    return flat(res)

In [210]:
slide_with_flank('abc','01234abc45678abc1234',up_flank=6,down_flank=6)

['bc1', 'c12', '123', '234']

In [195]:
def filter_with_identity_affinity(seq,full,identity_cutoff=0.5,step=1,up=6,down=6,flags=0):
    filtered = {}
    slides = slide_with_flank(seq,full,step=step,up_flank=up,down_flank=down,flags=flags)
    for s in slides:
        identity_score = calc_identity_score(seq,s)
        if identity_score <= identity_cutoff:
            #if s in filtered.keys(): #已经有score, 寸大值表明有大片段
            filtered[s] = identity_score
    return filtered

In [ ]:
for i in range(0,len(grist),10000):
    g = grist.iloc[i]
    if g['uniport_id'] in ids:
        full = str(uni_dict[g['uniport_id']].seq)
        res_local = filter_with_identity_affinity(g['peptide'],full)
        res_full = {}
        for win in res_local.keys(): # 过滤掉阳性集在window结果中 identity>0.5的
            for pep in np.random.choice(grist['peptide'],1000):#.remove(g['peptide']):
                score = calc_identity_score(win,pep)
                if score <0.5:
                    res_full[win] = score
                else:
                    break
        res_full_key = sorted(res_full.keys(),key=lambda x:(res_local[x],res_full[x]))
        grist.iloc[i]['neg1'] = res_full_key[0]

In [244]:
grist['neg1'] = np.full_like(grist['peptide'],'o')

In [ ]:
grist['neg1'] 

In [ ]:
grist['neg1'] = np.zeros

In [176]:
full = str(uni_dict['P62424'].seq)

In [193]:
seq=g['peptide']

In [177]:
slides = slide_with_flank(g['peptide'],full)

In [231]:
res = filter_with_identity_affinity(seq,full)

In [235]:
for i in p:
    print(i,res[i])

RPKNFGIGQDI 0.0
PKNFGIGQDIQ 0.0
WPRYIRLQRQR 0.0
PRYIRLQRQRA 0.0
RYIRLQRQRAI 0.0
YIRLQRQRAIL 0.0
IRLQRQRAILY 0.0
NQFTQALDRQT 0.0
YCIIKGKARLG 0.0
EAIRTNYNDRY 0.0
AIRTNYNDRYD 0.0
IRTNYNDRYDE 0.0
RTNYNDRYDEI 0.0
TNYNDRYDEIR 0.0
NYNDRYDEIRR 0.0
YNDRYDEIRRH 0.0
NDRYDEIRRHW 0.0
DRYDEIRRHWG 0.0
RYDEIRRHWGG 0.0
YDEIRRHWGGN 0.0
DEIRRHWGGNV 0.0
EIRRHWGGNVL 0.0
QDIQPKRDLTR 2.523234146875356e-18
GIGQDIQPKRD 0.018181818181818177
AINQFTQALDR 0.018181818181818177
FTQVNSEDKGA 0.018181818181818177
IGQDIQPKRDL 0.018181818181818184
FTQALDRQTAT 0.018181818181818184
TQVNSEDKGAL 0.018181818181818184
FEKRPKNFGIG 0.03636363636363637
TRFVKWPRYIR 0.03636363636363637
QALDRQTATQL 0.03636363636363637
LCRKMGVPYCI 0.03636363636363637
IIKGKARLGRL 0.03636363636363637
RLVHRKTCTTV 0.03636363636363637
LTRFVKWPRYI 0.03636363636363638
GRLVHRKTCTT 0.03636363636363638
IQPKRDLTRFV 0.054545454545454536
PYCIIKGKARL 0.054545454545454536
ETKQEKKQRLL 0.05454545454545454
DLTRFVKWPRY 0.05454545454545456
TKQEKKQRLLA 0.0545454545454545

In [213]:
persons.items()

dict_items([('ZhangSan', 'male'), ('LiSi', 'male'), ('WangHong', 'female')])

In [222]:
x= ['a','b','c']

In [218]:
remove

'b'

In [220]:
del x[1]

In [221]:
x

['a', 'c']

In [ ]:
gri